In [5]:
import pandas as pd
import geopandas as gpd
import numpy as np
import pathlib
import sys

In [3]:
raw_fold_drive = "G"

if sys.platform == 'darwin':
    print('mac')
    raw_fold = pathlib.Path('/users/ink/Google Drive/Shared drives/UGS_Flux/Data_Processing/Jupyter_Notebooks/cottonwood/')
else:
    print('pc')
    raw_fold = pathlib.Path(f'{raw_fold_drive}:/Shared drives/UGS_Flux/Data_Processing/Jupyter_Notebooks/cottonwood/')



pc


In [6]:
wellp = gpd.read_file('gis_files/well_siteid.gpkg')
#wellp = wellp[wellp['WIN']>0]
wellp['WIN'] = wellp['WIN'].astype(int)
wellp

,WIN,GROUP_NUMB,USES,GPM,WRCHEX,STATUS,LINK,Owner,Notes,Location,...,WebLink,FlowSource,SiteID,SiteDescri,Horsepower,Total_kWh,Totalizer,VibSensor_,PumpNotes,geometry
0,130,600375,A,0.0,71-5825,APPLWUC,https://waterrights.utah.gov/wellinfo/welldril...,NORTH SPRINGS LC,None,S1600 E515 S4 13 28S 11W SL,...,https://www.waterrights.utah.gov/search/?q=71-...,None,None,None,0.0,0.0,0.0,0.0,Likely Not being used or in wrong location,POINT (322137.466 4248123.478)
1,194,629454,I,254.0,71-118,UGWCCERT,https://waterrights.utah.gov/wellinfo/welldril...,WISEMAN FARMS LLC,None,N1469 E776 S4 28 28S 10W SL,...,https://www.waterrights.utah.gov/search/?q=71-118,None,None,None,0.0,0.0,0.0,0.0,None,POINT (327023.269 4245692.545)
2,210,603023,I,600.0,71-539,APPLAPP,https://waterrights.utah.gov/wellinfo/welldril...,BK YARDLEY LLC,None,N1330 E25 S4 19 28S 10W SL,...,https://www.waterrights.utah.gov/search/?q=a16522,None,None,None,0.0,0.0,0.0,0.0,None,POINT (323569.961 4247343.105)
3,2567,0,M,0.0,71-497,APPLCERT,https://waterrights.utah.gov/wellinfo/welldril...,CITY OF MILFORD,None,S821 W830 NE 07 28S 10W SL,...,https://www.waterrights.utah.gov/search/?q=71-...,None,None,None,0.0,0.0,0.0,0.0,None,POINT (324244.531 4251552.183)
4,2581,0,M,0.0,71-1227,APPLCERT,https://waterrights.utah.gov/wellinfo/welldril...,CITY OF MILFORD,None,S930 W275 NE 07 28S 10W SL,...,https://www.waterrights.utah.gov/search/?q=71-...,None,None,None,0.0,0.0,0.0,0.0,None,POINT (324413.695 4251518.96)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
521,0,634197,I,493.0,None,UGWC,None,NORTH SPRINGS LC,None,N170 W480 SE 35 28S 11W SL,...,https://www.waterrights.utah.gov/search/?q=71-...,None,None,None,0.0,0.0,0.0,0.0,None,POINT (320916.516 4243830.867)
522,0,628789,I,0.0,None,APPLWUC,None,JACKSON 3 L.L.C.,None,S1825 E1955 NW 07 29S 10W SL,...,https://www.waterrights.utah.gov/search/?q=71-...,None,None,None,50.0,0.0,0.0,0.0,None,POINT (323203.689 4241523.08)
523,0,603016,I,0.0,None,UGWC,None,ARLAN AND KRISTEN MAYER,None,N40 W930 E4 31 28S 10W SL,...,https://www.waterrights.utah.gov/search/?q=71-534,None,381954113004601,None,75.0,202909.0,0.0,0.0,difficult to read hp,POINT (324059.177 4244519.793)
524,0,603177,I,0.0,None,UGWC,None,B C FARMS FAMILY PARTNERSHIP LP,None,N40 E2410 W4 30 28S 10W SL,...,https://www.waterrights.utah.gov/search/?q=71-69,None,None,None,50.0,0.0,0.0,0.0,None,POINT (323537.31 4246182.063)


In [8]:
wlact = pd.read_csv('output_tables/wlactivity.txt')
wlact = wlact.drop_duplicates(subset=['activityType', 'win'])
wlact.set_index(['win', 'activityType'], inplace=True)
wlact = wlact['ACTIV_COM_DATE'].unstack(level=1)
wlact_dict = wlact[['New','Abandon']].to_dict()

In [11]:
import pathlib

wlfeat = pd.read_csv('output_tables/wlwellfeatures.txt',sep=',',
                     usecols=range(0,6))


wlfeat.set_index('win', inplace=True)
wlfeat['finishedDiameter'] = pd.to_numeric(wlfeat['finishedDiameter'], errors='coerce')
wlfeat['totalDepth'] = pd.to_numeric(wlfeat['totalDepth'], errors='coerce')
prop_dict = wlfeat[['finishedDiameter','totalDepth']].to_dict() 

In [12]:
wellp['finishedDiameter'] = wellp['WIN'].map(prop_dict['finishedDiameter'])
wellp['totalDepth'] = wellp['WIN'].map(prop_dict['totalDepth'])
wellp['NewWellDate'] = wellp['WIN'].map(wlact_dict['New'])
wellp['AbandonDate'] = wellp['WIN'].map(wlact_dict['Abandon'])
#wellp.to_csv('Milford_Pumping_2024_pi_enhanced.csv')

In [17]:
wellp.columns

Index(['well_id', 'SiteID', 'instflowgpm', 'horsepower', 'Site_kWh',
       'site_acft', 'sitehours', 'finishedDiameter', 'totalDepth',
       'NewWellDate', 'AbandonDate', 'proof_flow_gpm'],
      dtype='object')

In [15]:
rnm = {'WIN':'well_id',
       'Horsepower':'horsepower',
       'Inst_Flow_GPM':'instflowgpm',
       'SiteHours':'sitehours',
       'Site_acft':'site_acft',
       'Site_kWh':'total_kwh',}
wellp.rename(columns=rnm, inplace=True)
wellp['proof_flow_gpm'] = np.nan
wellp.replace(0, np.nan, inplace=True)

In [17]:
wellp.to_csv("output_tables/wells_with_properties.csv")

In [20]:
clean = wellp[["well_id","horsepower","proof_flow_gpm","instflowgpm","sitehours","site_acft","total_kwh"]]

# Derive a flow to use for hours->volume (prefers instflow, else proof)
clean["flow_gpm_for_hours"] = np.where(clean["instflowgpm"].notna() & (clean["instflowgpm"]>0),
                                       clean["instflowgpm"],
                                       clean["proof_flow_gpm"])

# Convert gpm to acft/hour
clean["flow_acft_per_hour"] = clean["flow_gpm_for_hours"] * 60.0 / 325851.0

# Naive volume estimates (sanity checks, not final): 
# from hours and from energy (assuming mid prior ~350 kWh/acft)
assumed_kwh_per_acft = 350.0
clean["vol_from_hours_acft"] = clean["flow_acft_per_hour"] * clean["sitehours"]
clean["vol_from_energy_acft"] = clean["total_kwh"] / assumed_kwh_per_acft

# Basic profiling
profile = {
    "rows": len(clean),
    "missing": clean.isna().sum().to_dict(),
    "nonzero_counts": (clean.notna() & (clean!=0)).sum().to_dict(),
    "hp_summary": clean["horsepower"].describe().to_dict(),
    "site_acft_summary": clean["site_acft"].describe().to_dict(),
    "energy_summary_kwh": clean["total_kwh"].describe().to_dict(),
    "hours_summary": clean["sitehours"].describe().to_dict(),
    "instflow_summary": clean["instflowgpm"].describe().to_dict(),
    "proof_flow_summary": clean["proof_flow_gpm"].describe().to_dict(),
}

# Save the prepared data snapshot
prep_path = pathlib.Path("pumping_prepared.csv")
clean.to_csv(prep_path, index=False)

/var/folders/jg/b66w1nm54nj99vk_9vx8vntr0000gn/T/ipykernel_25229/3566480235.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean["flow_gpm_for_hours"] = np.where(clean["instflowgpm"].notna() & (clean["instflowgpm"]>0),
/var/folders/jg/b66w1nm54nj99vk_9vx8vntr0000gn/T/ipykernel_25229/3566480235.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean["flow_acft_per_hour"] = clean["flow_gpm_for_hours"] * 60.0 / 325851.0
/var/folders/jg/b66w1nm54nj99vk_9vx8vntr0000gn/T/ipykernel_25229/3566480235.py:14: S